In [1]:
import pandas as pd
import numpy as np
import itertools

import logging, os

logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import tensorflow as tf
import matplotlib.pyplot as plt
import plotly.express as px

import time
import tqdm

ROOT_DIR = os.getcwd()
DATA_DIR = os.path.join(ROOT_DIR, 'data')


import json

In [2]:
ratings_list = [i.strip().split("::") for i in open(os.path.join(DATA_DIR,'ratings.dat'), 'r').readlines()]
users_list = [i.strip().split("::") for i in open(os.path.join(DATA_DIR,'users.dat'), 'r').readlines()]
movies_list = [i.strip().split("::") for i in open(os.path.join(DATA_DIR,'movies.dat'),encoding='latin-1').readlines()]
ratings_df = pd.DataFrame(ratings_list, columns = ['UserID', 'MovieID', 'Rating', 'Timestamp'])
movies_df = pd.DataFrame(movies_list, columns = ['MovieID', 'Title', 'Genres'])
movies_df['MovieID'] = movies_df['MovieID'].apply(pd.to_numeric)

print("Data loading complete!")
print("Data preprocessing...")

Data loading complete!
Data preprocessing...


In [3]:
movies_id_to_movies = {movie[0]: movie[1:] for movie in movies_list}
len(movies_list)

3883

In [4]:
genres = ['Action',
        'Adventure',
        'Animation',
        "Children's",
        'Comedy',
        'Crime',
        'Documentary',
        'Drama',
        'Fantasy',
        'Film-Noir',
        'Horror',
        'Musical',
        'Mystery',
        'Romance',
        'Sci-Fi',
        'Thriller',
        'War',
        'Western']

In [5]:
ratings_df.isnull().sum()

UserID       0
MovieID      0
Rating       0
Timestamp    0
dtype: int64

In [6]:
print(len(set(ratings_df["UserID"])) == max([int(i) for i in set(ratings_df["UserID"])]))
print(max([int(i) for i in set(ratings_df["UserID"])]))

True
6040


In [7]:
ratings_df = ratings_df.applymap(int)

C:\Users\samra\AppData\Local\Temp\ipykernel_37228\3375143496.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ratings_df = ratings_df.applymap(int)


In [8]:
ratings_df.head(5)

UserID  MovieID  Rating  Timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408       4  978300275
4       1     2355       5  978824291

In [9]:
users_dict = {user : [] for user in set(ratings_df["UserID"])}
users_dict[1]
ratings_df_gen = ratings_df.iterrows()
users_dict_for_history_len = {user : [] for user in set(ratings_df["UserID"])}
for data in ratings_df_gen:
    users_dict[data[1]['UserID']].append((int(data[1]['MovieID']),int(data[1]['Rating'])))
    if data[1]['Rating'] >= 4:
        users_dict_for_history_len[data[1]['UserID']].append((int(data[1]['MovieID']), int(data[1]['Rating'])))

In [10]:
users_history_lens = [len(users_dict_for_history_len[u]) for u in set(ratings_df["UserID"])]

In [11]:
users_num = max(ratings_df["UserID"])+1
items_num = max(ratings_df["MovieID"])+1

In [12]:
train_users_num = int(users_num * 0.8)
train_items_num = items_num
print(train_users_num, train_items_num)

4832 3953


In [13]:
train_users_dict = {int(k):users_dict[k] for k in range(1, train_users_num+1)}
train_users_history_lens = users_history_lens[:train_users_num]
print(len(train_users_dict),len(train_users_history_lens))

4832 4832


In [14]:
eval_users_num = int(users_num * 0.2)
eval_items_num = items_num
print(eval_users_num, eval_items_num)

1208 3953


In [15]:
eval_users_dict = {k:users_dict[k] for k in range(users_num-eval_users_num, users_num)}
eval_users_history_lens = users_history_lens[-eval_users_num:]
print(len(eval_users_dict),len(eval_users_history_lens))

1208 1208


In [16]:
with open('features/train_users_map.json', 'w', encoding='utf-8') as f:
    json.dump(train_users_dict, f, ensure_ascii=False, indent=4)

In [17]:
with open('features/eval_users_map.json', 'w', encoding='utf-8') as f:
    json.dump(eval_users_dict, f, ensure_ascii=False, indent=4)

In [18]:
ratings_df.to_csv('features/ratings.csv')
movies_df.to_csv('features/movies.csv')

In [19]:
data = {'ratings' : ratings_list , 'users' : users_list , 'movies':  movies_list}
with open('features/metadata.json', 'w', encoding='utf-8') as f:
    json.dump(data, f)

In [20]:
np.save('features/eval_users_history_lens.npy',np.array(eval_users_history_lens))
np.save('features/train_users_history_lens.npy',np.array(train_users_history_lens))